In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import re
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from bs4 import BeautifulSoup
import re
from random import randint
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pickle
import requests
import math
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import pyperclip 

In [2]:
with open('Other-Grantees-LinkedIn.pkl', 'rb') as f:
    others_batch = pickle.load(f)
others_batch.shape

(12061, 6)

In [3]:
batch = (6000,7000)

In [4]:
others_batch = others_batch.loc[batch[0]:batch[1]]
others_batch.shape

(1001, 6)

In [5]:
others_batch.head(1)

,Grantees,Index,Found Names,Found Links,Found Results,Searching Name
6000,"BRASWELL, BEVERLY D",8446,NaN,NaN,NaN,NaN


# Names Pool Light Search

In [6]:
browser = webdriver.Chrome('/Users/macbook/Downloads/Chrome Driver/chromedriver')
sleep(3)
url = 'https://www.linkedin.com'
browser.get(url)

In [7]:
browser = webdriver.Chrome('/Users/macbook/Downloads/Chrome Driver/chromedriver')
sleep(3)
url = 'https://www.linkedin.com'
browser.get(url)
sleep(3)
browser.find_element_by_xpath('/html/body/nav/a[3]').click()
sleep(4)
browser.find_element_by_xpath('//*[@id="app__container"]/main/div[2]/form//div/button').click()

In [24]:
def get_light_linkedin(start, end, data, file, instr_path):
    for i in range(start, end): 

        name = data['Grantees'].loc[i]

        new_name = re.sub(r'( TRUSTEE| LLC| EST| OF| TRUST| INC| LTDA)','', name)
        new_name = re.sub(r'(( \w)+$)','', new_name)
        new_name = re.sub(r'( \w )',' ', new_name)
        new_name = re.sub(r'(\, )',' ', new_name)

        data['Searching Name'].loc[i] = new_name

        if 'marriott' in new_name.lower():
            continue

        browser.find_element_by_xpath(instr_path).send_keys(new_name)

        browser.implicitly_wait(0.4)
        browser.find_element_by_xpath(instr_path).send_keys(Keys.ENTER)
        print(i, new_name)

        sleep(randint(1,2)) 
        actions = ActionChains(browser)
        for _ in range(3):
            actions.send_keys(Keys.PAGE_DOWN).perform()
            sleep(0.1)

        
        sleep(0.5)
        html = browser.page_source
        soup = BeautifulSoup(html, 'html.parser')

        sleep(0.1)
        found_names = []
        els = soup.findAll('span', attrs={'class':'name actor-name'})
        for a in els:
            found_names.append(a.text)

        if len(found_names) == 0:
            found_names='Not found'
            print('Not found')
        elif len(found_names) > 5:
            print(', '.join(found_names[:5]))
        else:
            print(', '.join(found_names))

        data['Found Names'].loc[i] = found_names

        sleep(0.1)
        found_links = []   
        els = soup.findAll('a', attrs={'class':'search-result__result-link ember-view'})
        for a in els:
            if a['href'] not in found_links:
                found_links.append(a['href'])
        if len(found_links) > 0:
            data['Found Links'].loc[i] = found_links

        # results
        try:
            results = browser.find_element_by_xpath(path_results).text
            print(results, '\n')
            data['Found Results'].loc[i] = results
        except:
            data['Found Results'].loc[i] = np.nan

        browser.find_element_by_xpath(instr_path).clear()

        if i%50==0 or i==end-1:
            sleep(randint(5,15))
            print(i,'Saved')
        with open(file, 'wb') as f:
            pickle.dump(data, f)
        
        # check limit
        try:
            limit_path = '/html/body/div[9]/div[4]/div/div/h1'
            limit = browser.find_element_by_xpath(limit_path).text
            if limit:
                print('Limit is reached')
                break
        except:
            pass

In [25]:
path_results = '/html/body/div[9]/div[4]/div/div[2]/div/div[2]/div/div/div/div/h3'
instr_path = '/html/body/header/div/form/div/div/div/div/div[1]/div[1]/input' 

In [26]:
file = 'Other-Grantees-LinkedIn-Batch_{}-{}.pkl'.format(batch[0], batch[1])
file

'Other-Grantees-LinkedIn-Batch_4000-5000.pkl'

In [ ]:
get_light_linkedin(batch[0]+2, batch[1], others_batch, file, instr_path)

4002 TAN ANNETTE
Annette Tan, Annette Tan, Annette Tan, Annette Tan, Annette Tan
Showing 126 results 

4003 ANNETTE TAN REVOCABLE LIVING
Not found
4004 BERTHOLD JUERGEN
Berthold Juergen, Juergen Berthold, Juergen Berthold, Juergen Berthold, Prof. Juergen Czarske
Showing 9 results 

4005 BERTHOLD ELIZABETH
Elizabeth Berthold, Elizabeth Berthold, Elizabeth Berthold, Elizabeth Hegedus-Berthold, Liz Berthold
Showing 18 results 

4006 HOROWITZ LEO
Leo Horowitz, Leo Horowitz, Leonard Horowitz, Leonard Horowitz, Leonard Horowitz
Showing 84 results 

4007 HOROWITZ MARCIA
Marcia Horowitz, Marcia Horowitz, Marcia Luxenburg-Horowitz, Marcia Horowitz, MARCIA HOROWITZ
Showing 22 results 

4008 NORTHENOR DENNIS
Dennis Northenor
Showing 1 result 

4009 NORTHENOR JOYCE
Not found
4010 SMITH TERRANCE
Terrance Smith, Terrance Smith, Terrance smith, Terrance D. Smith  MA Ed., Terrance Smith
Showing 1,314 results 

4011 RISLEY BONITA
Bonita Risley, Bonita Lock/Risley, Bonita Watkins
Showing 4 results 

401

4083 POWELL DAVID
Dave Powell, Dave Powell, David Powell, David Powell, David Powell
Showing 6,419 results 

4084 POWELL JOAN
Joan Powell, joan powell, Joan Powell, Joan Powell, Joan Powell
Showing 552 results 

4085 VLACH DIANE
Diane Vlach
Showing 2 results 

4086 VLACH JEFFREY
Jeff Vlach, Jeff Vlach, Jeff Vlach, Jeff Vlach, Jeff Vlach
Showing 11 results 

4087 KARRIKER DONALD
Don Karriker, Don Karriker, Eben Karriker
Showing 3 results 

4088 KARRIKER PATRICIA
Patricia Karriker
Showing 1 result 

4089 MORIN DALE
Dale Morin, Dale Morin, Dale M., Dale Morin, Dale M.
Showing 89 results 

4090 MORIN STANLEY
Stanley J. Morin, Stanley Morin MSN,RN,PCCN, Stanley Morin, Stan Morin, Stan Morin
Showing 110 results 

4091 RUGGIERO LISA
Lisa Ruggiero, Lisa Seigies, Lisa Ruggiero, Alisa (Lisa) Ruggiero, Lisa Ruggiero
Showing 141 results 

4092 RUGGIERO MICHAEL
Michael Ruggiero, Michael Ruggiero, Michael J. Ruggiero, Michael Ruggiero, Michael Ruggiero
Showing 409 results 

4093 CHILDRESS MALINDA
Me

4163 JONES GERALD
Jerry Jones, Gerald Jones, Jerry Jones, Jerry Jones, Gerald Jones
Showing 4,598 results 

4164 JONES LYNN
Lynn Jones, Lynn Jones, Lynn Jones, lynn jones, Lynn Jones
Showing 3,879 results 

4165 GONZALEZ TERESA
Tessa Pastor González, Teresa gonzalez, Teresa Gonzalez Rico, Teresa Gonzalez, Teresa Gonzalez
Showing 7,635 results 

4166 GONZALEZ RODRIGO
Rodrigo Gonzalez Argueta, Rodrigo Samayoa González, Rodrigo Gonzalez Lopez, Rodrigo Gonzalez, Rodrigo G.
Showing 7,540 results 

4167 ONETTO RAIMUNDO
Raimundo Onetto, Raimundo Onetto, Raimundo Onetto, Raimundo Onetto
Showing 4 results 

4168 DEMARIA MARCELA
Marcela Demaria, Marcela Demaria, Marcela Demaria, Marcela Viviana Demaria Baez, MARCELA MARIA DE ALMEIDA
Showing 59 results 

4169 KARCHNER JUDY
judith kirchner, Judy Karchner, Al & Judy Karchner, Judy Kirchner, Judy Kirchner-Davis
Showing 37 results 

4170 KARCHNER ALLAN
Allan Karchner, Allan Kirchner-Theriault (He/Him), allan kirchner
Showing 16 results 

4171 BAILEY 

4250 CHOPP GEORGE
George Chopp, george George Chopp, Brian M. Chopp, D.Sc., PMP
Showing 17 results 

4250 Saved
4251 GEORGE CHOPP REVOCABLE
Not found
4252 SNITKER JAMES
James Snitker, James Snitker, James Snitker, Jamie Snitker
Showing 11 results 

4253 WYLIE JANET
Janet Wylie, Janet Wylie, Janet Wylie, Janet Wylie, Janet Wylie
Showing 94 results 

4254 WILLIAMSON PHYLLIS
Phyllis Williamson, Phyllis Williamson, Phyllis Williamson, Phyllis Williamson, Phyllis Williamson
Showing 64 results 

4255 WILLIAMSON TERRY
Kevan Williamson, Joseph (Terry) Williamson, Michael Williamson, Terry Parker
Showing 675 results 

4256 DE NOVOA LORENA
Lorena Novoa, Lorena Novoa, lorena novoa, Lorena Novoa, Lorena Novoa
Showing 64 results 

4257 NOVOA Y MAXIMILIANO GUILLERMO
Not found
4258 TROIANO & ASSOCIATES
Domenic Troiano, FCPA, MIPA, CTA, RTA, AFA
Showing 71 results 

4259 TAYLOR LORRAINE
Lorraine Taylor, Lorraine T., Lorraine Taylor, Lorraine Taylor, Lorraine Taylor
Showing 952 results 

4260 TAYLOR RO

Showing 13 results 

4334 YABLUN JEFFREY
Jeff Yablun
Showing 1 result 

4335 MAGUIRE CARLA
Jeff Yablun
Showing 1 result 

4336 MAGUIRE MICHAEL
Carla Grosvenor, Carla Maguire, carla maguire, carla maguire, Maguire Carla
Showing 2,757 results 

4337 CALLAHAN RUBY
Michael Maguire, Michael Maguire, Michael Maguire, Michael Maguire, Michael Maguire
Showing 2,757 results 

4338 LUTHER ROBERT JR
Robert Luther Jr, Luther Elmore Jr., John Robert Leibforth Jr., Stanley Frencher Jr
Showing 1,084 results 

4339 LUTHER DEBORAH
Robert Luther Jr, Luther Elmore Jr., John Robert Leibforth Jr., Stanley Frencher Jr
Showing 1,084 results 

4340 WRIGHT LYNETTE
Lynette Wright, lynette wright, Lynette Wright, Lynette Wright, Lynette Wright
Showing 201 results 

4341 DEPEW DON
Lynette Wright, lynette wright, Lynette Wright, Lynette Wright, Lynette Wright
Showing 75 results 

4342 DEPEW LINDA
Linda Depew, Linda Depew, Linda DePew, Linda Depew, Linda Depew
Showing 87 results 

4343 DEPPEN JOHN
John Deppen, John

4414 ELLY YVONNE
Yvonne Schwemmer, Yvonne Buckerfield
Showing 19 results 

4415 ELLY ROBERT
Yvonne Schwemmer, Yvonne Buckerfield
Showing 207 results 

4416 RICKENBACH ROBERT
elly robert, Elly en Rob van Zuilen, Elly en Bob van amerongen, Rob/Elly Sanderson, Elly en Rob privé kambeel - van uffelen
Showing 207 results 

4417 RICKENBACH ROBIN
Not found
4418 DELPRIORE JOAN
Not found
4419 DELPRIORE TOM
Not found
Showing 4 results 

4420 HUESTON GEORGIANA
Tom Delpriore, Thomas Del Priore, Tom del priore, Thomas  L. Del Priore
Showing 4 results 

4421 HUESTON ROBERT
Tom Delpriore, Thomas Del Priore, Tom del priore, Thomas  L. Del Priore
Showing 4 results 

4422 HUESTON FAMILY
Tom Delpriore, Thomas Del Priore, Tom del priore, Thomas  L. Del Priore
Showing 4 results 

4423 CALIFANO JULIE
Tom Delpriore, Thomas Del Priore, Tom del priore, Thomas  L. Del Priore
Showing 4 results 

4424 CALIFANO LORENZO
Not found
4425 BABST JENNIFER
Not found
4426 BABST ROBERT
Not found
4427 PADDOCK JOSEPH
Not foun

In [29]:
others_batch.head(3)

,Grantees,Index,Found Names,Found Links,Found Results,Searching Name
4000,"SMITH, HOWARD J",7309,"[Howard Smith, Howard Smith]","[/in/howardasmith/, /in/howard1smith/]","Showing 14,086 results",SMITH HOWARD
4001,"TAN, FABER TRUSTEE",7310,"[Faber Tan, Faber Tan, Faber Tan, Kelvin Tan K...","[/in/faber-tan-89ba93a/, /in/faber-tan-4885437...",Showing 162 results,TAN FABER
4002,"TAN, ANNETTE TRUSTEE",7310,NaN,NaN,NaN,NaN
